In [1]:
import os
import shutil
import keras
import numpy as np
import random
from keras.applications import DenseNet121
from keras.optimizers import Adam
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, BatchNormalization, Dropout, Conv2D, MaxPooling2D, AveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping

# Define your dataset directory
# Define paths
dataset_dir = '/Users/sumlipuri/Desktop/plant_disease_detection-main/DataSet/color'  # Assuming your dataset is in this directory
train_dir = 'PlantVillage_newDataset/train'
test_dir = 'PlantVillage_newDataset/test'

class_names = os.listdir(dataset_dir)
num_classes = len(class_names)

# Create train and test directories
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)


# Define ratio for train-test split
train_ratio = 0.8  # 80% for training, 20% for testing

# Iterate over each class directory
for class_name in os.listdir(dataset_dir):
    class_dir = os.path.join(dataset_dir, class_name)
    if os.path.isdir(class_dir):
        # Create subdirectories in train and test directories
        os.makedirs(os.path.join(train_dir, class_name), exist_ok=True)
        os.makedirs(os.path.join(test_dir, class_name), exist_ok=True)
        
        # Get list of image files in class directory
        image_files = os.listdir(class_dir)
        # Shuffle the list of image files
        random.shuffle(image_files)
        
        # Split the image files into train and test sets
        num_train_samples = int(len(image_files) * train_ratio)
        train_images = image_files[:num_train_samples]
        test_images = image_files[num_train_samples:]
        
        # Copy train images to train directory
        for image_name in train_images:
            src = os.path.join(class_dir, image_name)
            dst = os.path.join(train_dir, class_name, image_name)
            shutil.copyfile(src, dst)
        
        # Copy test images to test directory
        for image_name in test_images:
            src = os.path.join(class_dir, image_name)
            dst = os.path.join(test_dir, class_name, image_name)
            shutil.copyfile(src, dst)

print("Dataset split into train and test sets.")




Dataset split into train and test sets.


In [5]:
dataset_dir = '/Users/sumlipuri/Desktop/plant_disease_detection-main/DataSet/color'  # Assuming your dataset is in this directory

# test_data_gen = ImageDataGenerator(rescale=1./255, validation_split=.2)
# train_data_gen = ImageDataGenerator(
#     rescale=1./255,
#     rotation_range=40,
#     width_shift_range=0.2,
#     height_shift_range=0.2,
#     shear_range=0.2,
#     zoom_range=0.2,
#     horizontal_flip=True,
#     fill_mode='nearest')

# train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

# # Load data
# train_data = train_datagen.flow_from_directory(
#     directory=dataset_dir,
#     target_size=(256, 256),
#     batch_size=32,
#     class_mode='categorical')

# test_data = test_data_gen.flow_from_directory(
#     directory=dataset_dir,
#     target_size=(256, 256),
#     batch_size=32,
#     class_mode='categorical')

# train_datagen = ImageDataGenerator(rescale=1./255)
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    validation_split = 0.2,
    horizontal_flip=True,
    fill_mode='nearest')

train_generator = train_datagen.flow_from_directory(
    dataset_dir,
    target_size=(256, 256),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    dataset_dir,
    target_size=(256, 256),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)


Found 43456 images belonging to 38 classes.
Found 10849 images belonging to 38 classes.


In [6]:

# Load pre-trained DenseNet121 model
model_d = DenseNet121(weights='imagenet', include_top=False, input_shape=(256, 256, 3))

# Add custom layers on top of the pre-trained model
x = model_d.output
x = GlobalAveragePooling2D()(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
x = Dense(1024, activation='relu')(x)
x = Dense(512, activation='relu')(x)
x = Dense(1024, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

# Output layer
preds = Dense(2, activation='softmax')(x) #change with number of classes

# Combine pre-trained model and custom layers
model = Model(inputs=model_d.input, outputs=preds)

# Freeze pre-trained layers for initial training
for layer in model_d.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.01), loss='categorical_crossentropy', metrics=['accuracy'])

# Data augmentation
train_data_gen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

test_data_gen = ImageDataGenerator(rescale=1./255)

# Load data
train_data = train_data_gen.flow_from_directory(
    directory="Users/sumlipuri/Desktop/plant_disease_detection-main/PlantVillage_newDataset/train/",
    target_size=(256, 256),
    batch_size=32,
    class_mode='categorical')

test_data = test_data_gen.flow_from_directory(
    directory="Users/sumlipuri/Desktop/plant_disease_detection-main/PlantVillage_newDataset/test/",
    target_size=(256, 256),
    batch_size=32,
    class_mode='categorical')

# Callbacks
checkpoint = ModelCheckpoint(
    "keras/models.keras",
    monitor='val_accuracy',
    verbose=1,
    save_best_only=True,
    save_weights_only=False,
    mode='auto')

early = EarlyStopping(
    monitor='val_accuracy',
    min_delta=0,
    patience=20,
    verbose=1,
    mode='max')

Found 52155 images belonging to 38 classes.
Found 19602 images belonging to 38 classes.


In [7]:
from keras.models import load_model
model = load_model('/Users/sumlipuri/Desktop/plant_disease_detection-main/plant_village_denseNet.h5')

In [8]:
accuracy = model.evaluate(test_data)
print('Test set accuracy:', accuracy[1] * 100)

613/613 [==============================] - 1770s 3s/step - loss: 0.0223 - accuracy: 0.9928
Test set accuracy: 99.28068518638611


In [9]:
from sklearn.metrics import classification_report, confusion_matrix
y_true = test_data.classes
y_pred = np.argmax(model.predict(test_data), axis=1)
print("\nClassification Report:")
print(classification_report(y_true, y_pred))

print("\nConfusion Matrix:")
print(confusion_matrix(y_true, y_pred))

613/613 [==============================] - 1626s 3s/step

Classification Report:
              precision    recall  f1-score   support

           0       0.02      0.02      0.02       230
           1       0.00      0.00      0.00       224
           2       0.00      0.00      0.00        97
           3       0.04      0.04      0.04       593
           4       0.03      0.03      0.03       533
           5       0.02      0.02      0.02       378
           6       0.01      0.01      0.01       312
           7       0.01      0.01      0.01       181
           8       0.01      0.01      0.01       441
           9       0.01      0.01      0.01       345
          10       0.02      0.02      0.02       417
          11       0.03      0.03      0.03       428
          12       0.02      0.02      0.02       500
          13       0.02      0.02      0.02       392
          14       0.00      0.00      0.00       151
          15       0.11      0.11      0.11      1995


In [11]:

print(confusion_matrix(y_true, y_pred))

[[  4   1   4 ...  25   1   9]
 [  3   0   0 ...  24   0   5]
 [  0   1   0 ...  12   0   5]
 ...
 [ 21  14   9 ... 193  12  53]
 [  3   1   1 ...  13   0   5]
 [ 11   4   2 ...  51   5  16]]


In [ ]:
from PIL import Image
filename = "D:/VSCode/AI & ML Project/Dataset/Apple/Apple___Apple_scab/0a5e9323-dbad-432d-ac58-d291718345d9___FREC_Scab 3417.JPG"
# AI & ML Project/Dataset/Tomato/Tomato___Bacterial_spot/00a7c269-3476-4d25-b744-44d6353cd921___GCREC_Bact.Sp 5807.JPG
# AI & ML Project/Dataset/Apple/Apple___Apple_scab/0a5e9323-dbad-432d-ac58-d291718345d9___FREC_Scab 3417.JPG
def process_image(image_path):
    # Load and preprocess the image for your model
    img = Image.open(image_path)
    img = img.resize((256, 256))  # adjust the size according to your model's input
    img_array = np.array(img) / 255.0  # normalize pixel values
    img_array = np.expand_dims(img_array, axis=0)  # add batch dimension
    return img_array

img_array = process_image(filename)
batch_prediction= model.predict(img_array)
print("Predicted Disease",class_names[np.argmax(batch_prediction[0])])